In [123]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import sklearn.metrics as metrics

%matplotlib inline

In [2]:
data = pd.read_csv('../../data/combined_kev.csv', parse_dates=['date'])

In [4]:
data

,date,species,wnvpresent,trap,latitude,longitude,species_ord,tmax,tmin,tavg,...,rel_humid_lag5,rel_humid_lag14,rel_humid_lag28,avgspeed_lag5,avgspeed_lag14,avgspeed_lag28,preciptotal_lag5,preciptotal_lag14,preciptotal_lag28,week_number
0,2007-05-29,CULEX PIPIENS,0,T096,41.731922,-87.677512,2.0,88,62,75,...,59.221367,40.692805,38.885597,7.51,9.682143,10.200000,1.500000e-01,0.069286,0.055357,22
1,2007-05-29,CULEX PIPIENS/RESTUANS,0,T086,41.688324,-87.676709,2.0,88,62,75,...,59.221367,40.692805,38.885597,7.51,9.682143,10.200000,1.500000e-01,0.069286,0.055357,22
2,2007-05-29,CULEX PIPIENS/RESTUANS,0,T048,41.867108,-87.654224,2.0,88,62,75,...,59.221367,40.692805,38.885597,7.51,9.682143,10.200000,1.500000e-01,0.069286,0.055357,22
3,2007-05-29,CULEX PIPIENS/RESTUANS,0,T129,41.891126,-87.611560,2.0,88,62,75,...,59.221367,40.692805,38.885597,7.51,9.682143,10.200000,1.500000e-01,0.069286,0.055357,22
4,2007-05-29,CULEX PIPIENS/RESTUANS,0,T050,41.919343,-87.694259,2.0,88,62,75,...,59.221367,40.692805,38.885597,7.51,9.682143,10.200000,1.500000e-01,0.069286,0.055357,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8470,2013-09-26,CULEX RESTUANS,0,T102,41.750498,-87.605294,1.0,75,52,64,...,57.732860,60.788648,65.361296,6.48,8.296429,8.271429,7.917278e-16,0.100714,0.088214,39
8471,2013-09-26,CULEX RESTUANS,0,T082,41.803423,-87.642984,1.0,75,52,64,...,57.732860,60.788648,65.361296,6.48,8.296429,8.271429,7.917278e-16,0.100714,0.088214,39
8472,2013-09-26,CULEX RESTUANS,0,T048,41.867108,-87.654224,1.0,75,52,64,...,57.732860,60.788648,65.361296,6.48,8.296429,8.271429,7.917278e-16,0.100714,0.088214,39
8473,2013-09-26,CULEX RESTUANS,0,T220,41.963976,-87.691810,1.0,75,52,64,...,57.732860,60.788648,65.361296,6.48,8.296429,8.271429,7.917278e-16,0.100714,0.088214,39


In [6]:
data.isnull().any().any()

False

In [7]:
data.duplicated().any()

False

In [9]:
# convert sunrise & sunset to datetime
sunrise = pd.to_datetime(data['sunrise'], format='%H%M')

# fix entries - round off 60min to the next hour
sunset = pd.to_datetime(data['sunset'].map(lambda x: 1800 if x == 1760 else x), format='%H%M')

# convert to seconds from start of day to be consistent input into model
data['sunrise'] = (sunrise - sunrise.dt.normalize()).dt.seconds
data['sunset'] = (sunset - sunset.dt.normalize()).dt.seconds

# add total sunlight time feature
data['total_sunlight_time'] = (sunset - sunrise).dt.seconds

In [48]:
X = data.drop(columns=['date', 'species', 'trap', 'wnvpresent'])
y = data['wnvpresent']

In [49]:
poly = PolynomialFeatures(include_bias=False, degree=2)
X_poly = poly.fit_transform(X)
X_poly.shape

(8475, 1274)

In [50]:
X_poly = pd.DataFrame(X_poly, columns=poly.get_feature_names(X.columns))

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X_poly,
                                                    y, 
                                                    random_state=42,
                                                    test_size=0.3, 
                                                    stratify=y)

In [52]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [53]:
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train_sc, y_train)

LogisticRegression(max_iter=10000)

In [54]:
metrics.roc_auc_score(y_test, lr.predict(X_test_sc))

0.5101176499141441

In [83]:
# Variable to store the optimum features
nof = 0           
high_score = 0

for i in [50, 100, 150, 250]:
    model = LogisticRegression(max_iter=10000)
    rfe = RFE(model, n_features_to_select=i, step=100)
    X_train_rfe = rfe.fit_transform(X_train_sc, y_train)
    X_test_rfe = rfe.transform(X_test_sc)
    model.fit(X_train_rfe, y_train)
    score = metrics.roc_auc_score(y_test, model.predict(X_test_rfe))
    if score > high_score:
        high_score = score
        nof = i
    print(f'n_features: {i}. Score: {score}')

print(f"Optimum number of features: {nof}")
print(f"Score with {nof} features: {high_score}")

n_features: 50. Score: 0.5030261936399877
n_features: 100. Score: 0.5030261936399877
n_features: 150. Score: 0.5101176499141441
n_features: 250. Score: 0.5101176499141441
Optimum number of features: 150
Score with 150 features: 0.5101176499141441


In [59]:
model = LogisticRegression(max_iter=10000)
rfe = RFE(model, n_features_to_select=10, verbose=2, step=100)
X_train_rfe = rfe.fit_transform(X_train_sc, y_train)
X_test_rfe = rfe.transform(X_test_sc)
model.fit(X_train_rfe, y_train)
score = metrics.roc_auc_score(y_test, model.predict(X_test_rfe))
print(score)

Fitting estimator with 1274 features.
Fitting estimator with 1174 features.
Fitting estimator with 1074 features.
Fitting estimator with 974 features.
Fitting estimator with 874 features.
Fitting estimator with 774 features.
Fitting estimator with 674 features.
Fitting estimator with 574 features.
Fitting estimator with 474 features.
Fitting estimator with 374 features.
Fitting estimator with 274 features.
Fitting estimator with 174 features.
Fitting estimator with 74 features.
0.5


In [ ]:

# Tabulating RFE results
rfe_results = [np.array(X_train.columns), rfe.ranking_]
rfe_results_df = pd.DataFrame(rfe_results).T
rfe_results_df.columns = ['Feature', 'RFE Ranking']

# Top 30 features
final_feature_list = rfe_results_df.loc[rfe_results_df['RFE Ranking'] == 1, 'Feature'].tolist()

In [166]:
logreg = LogisticRegression(
    solver='liblinear',
    max_iter=1000, 
    random_state=42,
)

logreg_params = {
    'clf__penalty': ['l1', 'l2'],
    'clf__C': [0.1, 1, 1.5, 2.5]
}

In [167]:
# Gradient Boost
gb = GradientBoostingClassifier(random_state=42)

gb_params = {
    'clf__learning_rate': [0.05, 0.1],
    'clf__max_depth': [2, 3]
}

In [168]:
# Support Vector Machine
svc = SVC(probability=True, random_state=42)

svc_params = {
    'clf__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'clf__C': [2, 5, 10]
}

In [ ]:
results = []

In [164]:
# Sorting by correlation to y
corr_df = X_poly.join(y).corr()[['wnvpresent']].abs().sort_values(by='wnvpresent', ascending=False)

# Instantiate result list
results = []

def model(clf, clf_params, cutoff):
    
    # Feature list with correlation to y > cutoff value
    features = corr_df[corr_df['wnvpresent'] > cutoff].index[1:]
    
    # Instantiate pipeline
    pipe = Pipeline([
        ('ss', StandardScaler()),
        ('clf', clf)
    ])
    
    # Gridsearch for best estimator
    grid = GridSearchCV(
        pipe,
        param_grid=clf_params,
        scoring='roc_auc',
        verbose=2,
        n_jobs=-1
    )

    grid.fit(X_train[features], y_train)
    
    print(f'Classifier: {clf}, Cutoff value: {cutoff}')
    print('Best Parameters:')
    print(grid.best_params_)
    
    # Scoring metrics
    scores = {'Classifier': clf, 'Cutoff': cutoff}
    y_preds = grid.predict(X_test[features])
    y_pred_probas = grid.predict_proba(X_test[features])[:, 1]
    y_train_pred_probas = grid.predict_proba(X_train[features])[:, 1]
    scores['Train ROC-AUC Score'] = metrics.roc_auc_score(y_train, y_train_pred_probas)
    scores['Test ROC-AUC Score'] = metrics.roc_auc_score(y_test, y_pred_probas)
    scores['F1'] = metrics.f1_score(y_test, y_preds)
    scores['Precision'] = metrics.precision_score(y_test, y_preds)
    scores['Recall'] = metrics.recall_score(y_test, y_preds)
    scores['Accuracy'] = metrics.accuracy_score(y_test, y_preds)
    
    # Storing results
    results.append(scores)
    
    return pipe

In [171]:
classifiers = [
    (logreg, logreg_params),
    (gb, gb_params),
    (svc, svc_params)
]

In [172]:
for cutoff in [0, 0.01, 0.05, 0.1]:
    for (clf, clf_params) in classifiers:
        model(clf, clf_params, cutoff)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Classifier: LogisticRegression(max_iter=1000, random_state=42, solver='liblinear'), Cutoff value: 0
Best Parameters:
{'clf__C': 1.5, 'clf__penalty': 'l1'}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Classifier: GradientBoostingClassifier(random_state=42), Cutoff value: 0
Best Parameters:
{'clf__learning_rate': 0.1, 'clf__max_depth': 2}
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Classifier: SVC(probability=True, random_state=42), Cutoff value: 0
Best Parameters:
{'clf__C': 5, 'clf__kernel': 'linear'}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Classifier: LogisticRegression(max_iter=1000, random_state=42, solver='liblinear'), Cutoff value: 0.01
Best Parameters:
{'clf__C': 2.5, 'clf__penalty': 'l1'}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Classifier: GradientBoostingClassifier(random_state=42), Cutoff value: 0.01
Best Parameters:
{'clf__learning_rate': 0.1, 'c

C:\Users\rifqi\anaconda3\envs\dsi\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classifier: GradientBoostingClassifier(random_state=42), Cutoff value: 0.1
Best Parameters:
{'clf__learning_rate': 0.1, 'clf__max_depth': 3}
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Classifier: SVC(probability=True, random_state=42), Cutoff value: 0.1
Best Parameters:
{'clf__C': 5, 'clf__kernel': 'rbf'}


C:\Users\rifqi\anaconda3\envs\dsi\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [173]:
pd.DataFrame(results).sort_values(by='Test ROC-AUC Score', ascending=False).reset_index(drop=True)

,Classifier,Cutoff,Train ROC-AUC Score,Test ROC-AUC Score,F1,Precision,Recall,Accuracy
0,GradientBoostingClassifier(random_state=42),0.05,0.884891,0.855815,0.040541,0.272727,0.021898,0.944160
1,"LogisticRegression(max_iter=1000, random_state...",0.01,0.880634,0.854353,0.041379,0.375000,0.021898,0.945340
2,"LogisticRegression(max_iter=1000, random_state...",0.00,0.878991,0.854212,0.041667,0.428571,0.021898,0.945733
3,"LogisticRegression(max_iter=1000, random_state...",0.00,0.878991,0.854212,0.041667,0.428571,0.021898,0.945733
4,GradientBoostingClassifier(random_state=42),0.01,0.889833,0.852942,0.054054,0.363636,0.029197,0.944947
5,GradientBoostingClassifier(random_state=42),0.00,0.891547,0.851727,0.066667,0.384615,0.036496,0.944947
6,GradientBoostingClassifier(random_state=42),0.00,0.891547,0.851727,0.066667,0.384615,0.036496,0.944947
7,"LogisticRegression(max_iter=1000, random_state...",0.05,0.875905,0.849338,0.013986,0.166667,0.007299,0.944554
8,GradientBoostingClassifier(random_state=42),0.10,0.924722,0.841508,0.169697,0.500000,0.102190,0.946127
9,"LogisticRegression(max_iter=1000, random_state...",0.10,0.855964,0.832854,0.000000,0.000000,0.000000,0.946127


In [117]:
# Selecting by correlation to y
corr_df = X_poly.join(y).corr()[['wnvpresent']].abs().sort_values(by='wnvpresent', ascending=False)

# Variable to store best cutoff point & ROC-AUC score
best_cutoff = 0
best_score = 0

# Check for best cutoff point
for i in np.arange(0, 0.11, 0.01):
    features = corr_df[corr_df['wnvpresent'] > i].index[1:]
    logreg = LogisticRegression(max_iter=10000)
    ss = StandardScaler()
    X_train_sc = ss.fit_transform(X_train[features])
    X_test_sc = ss.transform(X_test[features])
    logreg.fit(X_train_sc, y_train)
    score = metrics.roc_auc_score(y_test, logreg.predict(X_test_sc))
    if score >= best_score:
        best_cutoff = i
        best_score = score
    print(f'Cutoff Point: {i}, ROC_AUC Score: {score}')
    
print(f'Best cutoff: {best_cutoff}')

Cutoff Point: 0.0, ROC_AUC Score: 0.5101176499141441
Cutoff Point: 0.01, ROC_AUC Score: 0.5101176499141441
Cutoff Point: 0.02, ROC_AUC Score: 0.5028183798411514
Cutoff Point: 0.03, ROC_AUC Score: 0.5028183798411514
Cutoff Point: 0.04, ROC_AUC Score: 0.5028183798411514
Cutoff Point: 0.05, ROC_AUC Score: 0.5028183798411514
Cutoff Point: 0.06, ROC_AUC Score: 0.5028183798411514
Cutoff Point: 0.07, ROC_AUC Score: 0.4997921862011638
Cutoff Point: 0.08, ROC_AUC Score: 0.5
Cutoff Point: 0.09, ROC_AUC Score: 0.5
Cutoff Point: 0.1, ROC_AUC Score: 0.5
Best cutoff: 0.01


In [178]:
corr_df[corr_df['wnvpresent'] > 0.01].head(40)

,wnvpresent
wnvpresent,1.000000
sunrise total_sunlight_time,0.175753
sunrise rel_humid_lag14,0.165340
rel_humid_lag14 week_number,0.161684
rel_humid_lag5 week_number,0.158963
sunrise rel_humid_lag5,0.153953
sunrise tavg_lag14,0.152279
tavg_lag28 rel_humid_lag14,0.151942
sunrise tavg_lag28,0.148133
species_ord rel_humid_lag14,0.148097


In [113]:
np.arange(0, 0.21, 0.01)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ])

In [74]:
q = X_poly.join(y).corr()[['wnvpresent']].abs().sort_values(by='wnvpresent', ascending=False)

In [108]:
print(q[q['wnvpresent'] > 0.01].index[1:])

Index(['sunrise total_sunlight_time', 'sunrise rel_humid_lag14',
       'rel_humid_lag14 week_number', 'rel_humid_lag5 week_number',
       'sunrise rel_humid_lag5', 'sunrise tavg_lag14',
       'tavg_lag28 rel_humid_lag14', 'sunrise tavg_lag28',
       'species_ord rel_humid_lag14', 'tmin week_number',
       ...
       'sealevel HZ', 'resultdir tavg_lag5', 'HZ^2', 'HZ', 'latitude HZ',
       'avgspeed week_number', 'sunset preciptotal_lag28',
       'DZ rel_humid_lag28', 'tmax resultdir', 'cool preciptotal'],
      dtype='object', length=816)


In [142]:
logreg = LogisticRegression(max_iter=10000)
sss = StandardScaler()
logreg.fit(sss.fit_transform(X_train[q[q['wnvpresent'] > 0.05].index[1:]]), y_train)

LogisticRegression(max_iter=10000)

In [144]:
metrics.roc_auc_score(y_test, logreg.predict(sss.transform(X_test[q[q['wnvpresent'] > 0.05].index[1:]])))

0.5028183798411514

In [93]:
y_test.value_counts()

0    2406
1     137
Name: wnvpresent, dtype: int64

In [96]:
print(pd.options.display.max_columns)

20


In [98]:
print(pd.options.display.max_rows)

60


In [99]:
pd.options.display.max_columns = 200

In [153]:
metrics.roc_auc_score(y_test, logreg.predict_proba(sss.transform(X_test[q[q['wnvpresent'] > 0.05].index[1:]]))[:, 1])

0.8472962969704693